# Web Scraping with Beautiful Soup

Lauren F. Klein wrote version 1.0 of this notebook, based on lessons by [Alison Parrish](http://www.decontextualize.com/) and [Jinho Choi](https://github.com/emory-courses/data-science/blob/master/course/data_aggregation/data_aggregation.ipynb), which I have supplemented with material written by [Melanie Walsh](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Collecting-Cultural-Data/Web-Scraping.html)

*Quick discussion of plaforms (again) and file formats. We'll leave character encoding to next class.*

[Download relevant files here](https://melaniewalsh.org/Web-Scraping.zip)

## Why Do We Need To Scrape At All?

To perform practical approaches to data science with text, we need...**text**. Some text is prepared for computational analysis and publicly available in digital libraries. We can easily get novels in the public domain as .txt files, for example, from [Project Gutenberg](https://www.gutenberg.org/). Or we can work with text from HathiTrust's vast collections through the [HathiTrust Research Center](https://analytics.hathitrust.org/). 

If we want to work with text from popular culture and the internet--and we do!--we need to scrape the web. To understand the necessity and significance of web scraping, let's walk through the likely data collection process behind [“Film Dialogue from 2,000 screenplays, Broken Down by Gender and Age”](https://pudding.cool/2017/03/film-dialogue/) or any project similar to it.

One of the biggest sources for *The Pudding*'s screenplay data was the [Cornell Movie Dialogues Corpus](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). This is a corpus created by Cornell CIS professors Cristian Danescu-Niculescu-Mizil and Lillian Lee for their paper ["Chameleons in imagined conversations"](http://www.cs.cornell.edu/~cristian/papers/chameleons.pdf). These researchers helpfully shared a dataset of every URL that they used to find and access the screenplays in their own project.

Let's take a look:

First we import pandas, a Python library for managing data

In [43]:
import pandas as pd

Then, we read in the data, which is stored as a .csv file, and which you should have downloaded at the beginning of this notebook. You'll see three "arguments" between parentheses in the code below. The first is the filepath, telling pandas where to find the data. You might have to edit this argument to match where you've stored the data on your computer. Don't worry about the next two arguments for now.

In [44]:
urls = pd.read_csv("../data/cornell-movie-corpus/raw_script_urls.csv", delimiter='\t', encoding='utf=8')

Now, when we run the code naming the data we've just read in, Jupyter will render the top and bottom five rows from the .csv for us.

In [45]:
urls

,id,movie_title,script_url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt
...,...,...,...
612,m612,watchmen,http://www.scifiscripts.com/scripts/wtchmn.txt
613,m613,xxx,http://www.dailyscript.com/scripts/xXx.txt
614,m614,x-men,http://www.scifiscripts.com/scripts/xmenthing...
615,m615,young frankenstein,http://www.horrorlair.com/scripts/young.txt


### Inspecting HTML's anatomy with Developer Tools

Thanks to Alison Parrish, we have a very simple example of HTML to begin with. It is about kittens. [Here's the rendered version](http://static.decontextualize.com/kittens.html), and [here's the HTML source code](https://raw.githubusercontent.com/ledeprogram/courses/master/databases/data/kittens.html).

First we're going to use Developer Tools in Chrome to take a look at how `kittens.html` is organized. Click on the "rendered version" link above. In Chrome, ctrl-click (or right click) anywhere on the page and select "Inspect Element." This will open Chrome's Developer Tools. Your screen should look (something) like this:

<a href="http://static.decontextualize.com/snaps/kittens-dev-tools.png"><img src="http://static.decontextualize.com/snaps/kittens-dev-tools.png" alt="kittens-dev-tools"/></a>

In the upper panel, you see the web page you're inspecting. In the lower panel, you see a version of the HTML source code, with little arrows next to some of the lines. (The little arrows allow you to collapse parts of the HTML source that are hierarchically related.) As you move your mouse over the elements in the top panel, different parts of the source code will be highlighted. Chrome is showing you which parts of the source code are causing which parts of the page to show up. Pretty spiffy!

This relationship also works in reverse: you can move your mouse over some part of the source code in the lower panel, which will highlight in the top panel what that source code corresponds to on the page. We'll be using this later to visually identify the parts of the page that are interesting to us, so we can write code that extracts the contents of those parts automatically.

### Characterizing the structure of kittens

Here's what the source code of kittens.html looks like:

	<!doctype html>
	<html>
	  <head>
	    <title>Kittens!</title>
	  </head>
	  <body>
	    <h1>Kittens and the TV Shows They Love</h1>
	    <div class="kitten">
	      <h2>Fluffy</h2>
	      <div><img src="http://placekitten.com/100/100"></div>
	      <ul class="tvshows">
	        <li><a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a></li>
	        <li><a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a></li>
	      </ul>
	      Last check-up: <span class="lastcheckup">2014-01-17</span>
	    </div>
	    <div class="kitten">
	      <h2>Monsieur Whiskeurs</h2>
	      <div><img src="http://placekitten.com/150/100"></div>
	      <ul class="tvshows">
	        <li><a href="http://www.imdb.com/title/tt0106179/">The X-Files</a></li>
	        <li><a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a></li>
	      </ul>
	      Last check-up: <span class="lastcheckup">2013-11-02</span>
	    </div>
	  </body>
	</html>

This is pretty well organized HTML, but if you don't know how to read HTML, it will still look like a big jumble. Here's how I would characterize the structure of this HTML, reading in my own idea of what the meaning of the elements are.

* We have two "kittens," both of which are contained in `<div>` tags with class `kitten`.
* Each "kitten" `<div>` has an `<h2>` tag with that kitten's name.
* There's an image for each kitten, specified with an `<img>` tag.
* Each kitten has a list (a `<ul>` with class `tvshows`) of television shows, contained within `<li>` tags.
* Those list items themselves have links (`<a>` tags) with an `href` attribute that contains a link to an IMDB entry for that show.

**SOME HTML QUESTIONS FOR YOU:**
* What's the parent tag of `<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>`? 

* Both `<div class="kitten">` tags share a parent tag---what is it? What attributes are present on both `<img>` tags?

### Scraping kittens with Beautiful Soup

We've examined `kittens.html` a bit now. What we'd like to do is write some code that is going to extract information from the HTML, like "what is the last checkup date for each of these kittens?" or "what are Monsieur Whiskeur's favorite TV shows?" To do so, we need to *parse* the HTML, and create a representation of it in our program that we can manipulate with Python.

As mentioned earlier, HTML is hard to parse by hand. (Don't even try it. In particular, [don't parse HTML with regular expressions](http://stackoverflow.com/a/1732454).)

[Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/) is a Python library that parses HTML (even if it's poorly formatted) and allows us to extract and manipulate its contents. More specifically, it gives us some Python objects that we can call methods on to poke at the data contained therein. So instead of working with strings and bytes, we can work with Python objects, methods, and data structures.

If you're using Anaconda, Beautiful Soup comes pre-installed, so there's no extra installation step! (We will learn how to install new libraries in future classes). 

Note that Beautiful Soup only *parses* HTML. It's left to us to actually *get* the HTML from somewhere. In most cases, we'll want to download the HTML directly from the actual web. For that, we'll use the `get` method from the Python library `requests` ([link](https://2.python-requests.org/en/master/)):

**Side note:** Since you're familiar with R, you know what a function is: a block of organized, reusable code that is called by a name, and performs some sort of action. Python has functions too, as well as things called methods, which are functions that are associated only with a particular object or class. Keep on reading to see an example of the 
`get` method in action.

In [ ]:
# first let's import the requests library
import requests 

If it worked, you won't get an error message. You will see that the execution counter has been incremented by 1.

Now let's use the "get" method to make an http request (the eponymous "requests") to get the contents of kittens.html.

In [ ]:
resp = requests.get("http://static.decontextualize.com/kittens.html") 

Note that "resp" is a Python object, and not plain text.

Also note that the "get" method makes things easy by guessing at the document's character encoding. 

We're not really going to talk about character encoding until next class, but since we can, let's check this page's encoding real quick.

In [ ]:
resp.encoding

Now let's create a string with the contents of the web page in text format we use the "text" method for this.

In [ ]:
html_str = resp.text

Let's see what it looks like:

In [ ]:
html_str

That looks like a mess but it's apparent that we've obtained the data as desired.

Now we need to create a Beautiful Soup object from that data. Here's how to go about doing that:

In [ ]:
from bs4 import BeautifulSoup

document = BeautifulSoup(html_str, "html.parser")
type(document)

The `BeautifulSoup` function creates a new Beautiful Soup object. It takes two parameters: the string containing the HTML data, and a string that designates which underlying parser to use to build the parsed version of the document. (Leave this as `"html.parser"`.) I've assigned this object to the variable `document`. This object supports a number of interesting methods that allow us to dig into the contents of the HTML. Primarily what we'll be working with are `Tag` objects and `ResultSet` objects, which are essentially just lists of `Tag` objects.

### Finding a tag

As we've previously discussed, HTML documents are composed of tags. To represent this, Beautiful Soup has a type of value that represents tags. We can use the `.find()` method of the `BeautifulSoup` object to find a tag that matches a particular tag name. For example:

In [ ]:
h1_tag = document.find('h1')
type(h1_tag)

A `Tag` object has several interesting attributes and methods. The `string` attribute of a `Tag` object, for example, returns a string representing that tag's contents:

In [ ]:
h1_tag.string

You can access the attributes of a tag by treating the tag object as though it were a dictionary, using the square-bracket index syntax, with the name of the attribute whose value you want as a string inside the brackets. 

**NOTE FOR FOLKS USING PYTHON FOR THE FIRST TIME:**

Unlike R, which (I think) just has lists, Python has two data types that store values: lists which are ordered, and store only one value, and dictionaries--what I'm talking
about here--that store UNORDERED sets of values in key:value pairs. 

For more on dictionaries, see [this notebook](dictionaries-sets-tuples.ipynb).

For now, though, just see if you can follow this example:

To print out the `src` attribute of the first `<img>` tag in the document, you would write:

In [ ]:
img_tag = document.find('img')

img_tag['src']

**Note:** You might have noticed that there is more than one `<img>` tag in `kittens.html`! If more than one tag matches the name you pass to `.find()`, it returns only the *first* matching tag. (A better name for `.find()` might be `find_first`.)

### Finding multiple tags

It's very often the case that we want to find not just one tag that matches particular criteria, but ALL tags matching those criteria. For that, we use the `.find_all()` method of the `BeautifulSoup` object. For example, to find all `h2` tags in the document:

In [ ]:
h2_tags = document.find_all('h2')
type(h2_tags)

But what's in the Result Set?

In order to find out, we're going to need to use a loop.

We're going to use some 'for' loop syntax that's very common in Python:

In [ ]:
for tag in h2_tags:
    print(tag.string) 

This conventiently gives you a variable, `tag`, that updates with the appropriate value each time you iterate. 

Does R have this too? You tell me! 

(For more on loops and counting, see [this notebook](counting.ipynb)). 

In any case, both the `.find()` and `.find_all()` methods can search not just for tags with particular names, but also for tags that have particular attributes. For that, we use the `attrs` keyword argument, giving it a dictionary that associates attribute names as keys and the desired attribute value as values. For example, to find all `span` tags with a `class` attribute of `lastcheckup`:

In [ ]:
checkup_tags = document.find_all('span', attrs={'class': 'lastcheckup'})
[tag.string for tag in checkup_tags]

**Important note about list comprehension in Python**

Line 2 above is a helpful shorthand: it creates a list (same as R lists) with each of the `tag.string`s in `checkup_tags`.

In more official terms, it's called a *list comprehension*, and it helps with a very common task in both data analysis and computer programming: when you want to apply an operation to every item in a list (e.g., scaling the numbers in a list by a fixed factor), or create a copy of a list with only those items that match a particular criterion (e.g., eliminating values that fall below a certain threshold). 

A list comprehension has a few parts:

* a source list, or the list whose values will be transformed or filtered;
* a predicate expression, to be evaluated for every item in the list;
* (optionally) a membership expression that determines whether or not an item in the source list will be included in the result of evaluating the list comprehension, based on whether the expression evaluates to True or False; and
* a temporary variable name by which each value from the source list will be known in the predicate expression and membership expression.
These parts are arranged like so:

> `[` *predicate expression* `for` *temporary variable name* `in` *source list* `if` *membership expression* `]`

The words for, in, and if are a part of the syntax of the expression. They don't mean anything in particular (and in fact, they do completely different things in other parts of the Python language). You just have to spell them right and put them in the right place in order for the list comprehension to work.

You can see more examples of this in action [here](lists.ipynb).

**An unrelated note, but before we move on:**

Beautiful Soup's `.find()` and `.find_all()` methods are actually more powerful than we're letting on here. [Check out the details in the official Beautiful Soup documentation.](http://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all)